In [18]:
from git import Repo, RemoteProgress
import os
import subprocess

In [19]:
REPO_URL_START = "https://github.com/"
REPO_URL_END = ".git"
REPO_CLONE_PATH = "../repos/"

In [20]:
# TODO progress display: https://stackoverflow.com/questions/38861829/how-do-i-implement-a-progress-bar

In [23]:
# https://gitpython.readthedocs.io/en/stable/reference.html
class LocalRepo:
    def __init__(self, name):
        self.name = name
        
    def update(self):
        if not self.is_cloned():
            print("cloning " + self.name + ", this may take a while...")
            self.clone()
        else:
            print("updating " + self.name + "...")
            self.pull()
        print("Repo is up to date!")
    
    def pull(self):
        # https://github.com/gitpython-developers/GitPython/issues/296#issuecomment-449769231
        Repo(self.path()).remote().fetch("+refs/heads/*:refs/heads/*")
    
    def clone(self):
        Repo.clone_from(self.url(), self.path(), None, None, ["--bare"])
    
    def is_cloned(self):
        return os.path.isdir(self.path())
    
    def path(self):
        return REPO_CLONE_PATH + self.name
    
    def url(self):
        return REPO_URL_START + self.name + REPO_URL_END
    
    def type_extension(self):
        return "java" # return the file extension that the files of your language have
    
    def get_file_objects(self):
        ending = "." + self.type_extension()
        files = []
        for git_object in Repo(self.path()).head.commit.tree.traverse():
            if git_object.type == "blob":
                if git_object.name.endswith(ending):
                    files.append(git_object)
        return files
    
    def get_file_object_content(self, git_object):
        return git_object.data_stream.read()


In [ ]:
class RepoTree:
    
    @staticmethod
    def init_from_repo(repo):
        found_nodes = RepoTree("")
        files = repo.get_file_objects()
        print("Analyzing " + str(len(files)) + " files...")
        for file_obj in files:
            file = RepoFile(r, file_obj)
            def handle(logic_path, node):
                found_nodes.register(logic_path)
            file.walk_tree(handle)
        print("Found " + str(found_nodes.node_count()) + " classes, methods and fields!")

        with open("../debug-tree.json", "w") as outfile:
            outfile.write(found_nodes.to_json())
    
    def __init__(self, name):
        self.name = name
        self.children = {}
    
    def register(self, path):
        self.register_list(path.split("/"))
        
    def register_list(self, path_segments):
        if len(path_segments) > 0:
            self.get_child(path_segments[0]).register_list(path_segments[1:])
    
    def get_child(self, name):
        if not name in self.children:
            self.children[name] = RepoTree(name)
        return self.children[name]
    
    def to_json(self):
        if len(self.children) == 0:
            return '{"name":"' + self.name + '"}'
        else:
            child_json = ",".join([c.to_json() for c in self.children.values()])
            return '{"name":"' + self.name + '","children":[' + child_json + ']}'
    
    def node_count(self):
        return sum([c.node_count() for c in self.children.values()]) + 1